# 0. Setup

In [2]:
import pytorch3d as p3d
import torch
from starter.utils import get_mesh_renderer

/root/anaconda3/envs/ml3d/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
renderer = get_mesh_renderer(image_size=512)

In [3]:
vertices = torch.tensor([[[0.0, 0.0, 0.0],
                          [1.0, 0.0, 0.0],
                          [0.0, 1.0, 0.0]]])  # 1 x N_v x 3 tensor.
faces = torch.tensor([[[0, 1, 2]]])  # 1 x N_f x 3 tensor.
textures =  torch.tensor([[[1.0, 0.0, 0.0],
                             [0.0, 1.0, 0.0],
                             [0.0, 0.0, 1.0]]])  # 1 x N_v x 3 tensor.
meshes = p3d.structures.Meshes(
    verts=vertices,
    faces=faces,
    textures=p3d.renderer.TexturesVertex(textures),
)

In [4]:
cameras = p3d.renderer.FoVPerspectiveCameras(
    R=torch.eye(3).unsqueeze(0),
    T=torch.tensor([[0.0, 0.0, 3.0]]),
    fov=60.0,
)

In [5]:
lights = p3d.renderer.PointLights(location=[[0.0, 0.0, -3.0]])
rend = renderer(meshes, cameras=cameras, lights=lights)
image = rend[0, ..., :3].numpy()

# 1. Practicing with Cameras (15 Points)

In [46]:
import numpy as np
from tqdm import tqdm
import imageio
from starter.utils import get_mesh_renderer

def camera360(mesh, img_size=256, num_frames=36, duration=1000, distance=3.0, elevation=15.0, output_path=None):
    renderer = get_mesh_renderer(img_size)

    duration //= num_frames  # duration per frame in milliseconds
    degrees = np.linspace(0, 360, num=num_frames).tolist()
    my_images = []

    # lights = p3d.renderer.PointLights(location=[[0.0, 0.0, -3.0]])

    for angle in tqdm(degrees):
        R, T = p3d.renderer.cameras.look_at_view_transform(distance, elevation, angle)
        cameras = p3d.renderer.FoVPerspectiveCameras(R=R, T=T, fov=60.0)
        lights = p3d.renderer.PointLights(location=cameras.get_camera_center())
        rend = renderer(mesh, cameras=cameras, lights=lights)
        image = rend[0, ..., :3].numpy()
        my_images.append((image * 255).astype(np.uint8))

    if output_path is not None:
        imageio.mimsave(output_path, my_images, duration=duration, loop=0)
        return None
    else:
        return my_images

## 1.1. 360-degree Renders (5 points)

In [22]:
import pytorch3d as p3d

cow_mesh = p3d.io.load_objs_as_meshes(["data/cow.obj"])
camera360(cow_mesh, output_path="output/cow-360.gif")

  0%|          | 0/36 [00:00<?, ?it/s]

100%|██████████| 36/36 [00:28<00:00,  1.27it/s]


Result:

![360-degree renders of the cow](output/cow-360.gif)

## 1.2 Re-creating the Dolly Zoom (10 points)

In [ ]:
# python -m starter.dolly_zoom --duration 30 --num_frames 100

Result:

![Dolly Zoom Effect](output/dolly.gif)

# 2. Practicing with Meshes (10 Points)

## 2.1 Constructing a Tetrahedron (5 points)

In [ ]:
import pytorch3d as p3d
import torch

points = torch.tensor([[
    [0.0, 0.0, -1.0],
    [0.0, 0.0, 1.0],
    [1.0, 1.0, 1.0],
    [1.0, -1.0, 1.0],
]])
faces = torch.tensor([[
    [0, 1, 2],
    [0, 1, 3],
    [1, 2, 3],
    [0, 2, 3],
]])
textures = torch.tensor([[
    [1.0, 0.0, 0.0],
    [0.0, 1.0, 0.0],
    [0.0, 0.0, 1.0],
    [1.0, 0.0, 1.0],
]])
textures = textures[:, :, None, None, :]

tetrahedron = p3d.structures.Meshes(
    verts=points,
    faces=faces,
    textures=p3d.renderer.TexturesAtlas(textures)
)

camera360(tetrahedron, num_frames=100, duration=2000, output_path="output/tetrahedron.gif")

100%|██████████| 100/100 [00:01<00:00, 71.52it/s]


Result:

- Number of vertices: 4
- Number of faces: 4

![](output/tetrahedron.gif)

## 2.2 Constructing a Cube (5 points)

In [47]:
import pytorch3d as p3d
import torch

points = torch.tensor([[
    [-1.0, -1.0, -1.0],
    [-1.0, -1.0, 1.0],
    [-1.0, 1.0, -1.0],
    [-1.0, 1.0, 1.0],
    [1.0, -1.0, -1.0],
    [1.0, -1.0, 1.0],
    [1.0, 1.0, -1.0],
    [1.0, 1.0, 1.0],
]])
faces = torch.tensor([[
    [0, 4, 5],
    [0, 5, 1],
    [4, 6, 7],
    [4, 7, 5],
    [6, 2, 3],
    [6, 3, 7],
    [2, 0, 1],
    [2, 1, 3],
    [1, 5, 7],
    [1, 7, 3],
    [2, 6, 4],
    [2, 4, 0],
]])
textures = torch.tensor([[
    [1.0, 0.0, 0.0],
    [1.0, 0.0, 0.0],
    [0.0, 1.0, 0.0],
    [0.0, 1.0, 0.0],
    [0.0, 0.0, 1.0],
    [0.0, 0.0, 1.0],
    [1.0, 0.0, 1.0],
    [1.0, 0.0, 1.0],
    [1.0, 1.0, 0.0],
    [1.0, 1.0, 0.0],
    [0.0, 1.0, 1.0],
    [0.0, 1.0, 1.0],
]])
textures = textures[:, :, None, None, :]

cube = p3d.structures.Meshes(
    verts=points,
    faces=faces,
    textures=p3d.renderer.TexturesAtlas(textures)
)

camera360(cube, num_frames=100, duration=2000, distance=5, elevation=30, output_path="output/cube.gif")

100%|██████████| 100/100 [00:01<00:00, 65.14it/s]


Result:

- Number of vertices: 8
- Number of triangle faces: 12

![](output/cube.gif)

# 3. Re-texturing a mesh (10 points)

In [55]:
import pytorch3d as p3d
import torch
from starter.utils import load_cow_mesh

vertices, faces = load_cow_mesh('data/cow.obj')

def color_interpolate(color1, color2, alpha):
    return alpha * color2 + (1 - alpha) * color1

z_min = vertices[:, 2].min()
z_max = vertices[:, 2].max()

color1 = torch.tensor([0.0, 0.0, 1.0])  # Blue color
color2 = torch.tensor([1.0, 0.0, 0.0])  # Red color

alphas = (vertices[:, 2] - z_min) / (z_max - z_min)

textures = color_interpolate(color1[None, ...], color2[None, ...], alphas[..., None]).unsqueeze(0)
vertices = vertices.unsqueeze(0)
faces = faces.unsqueeze(0)

cow_mesh = p3d.structures.Meshes(
    verts=vertices,
    faces=faces,
    textures=p3d.renderer.TexturesVertex(textures)
)

camera360(cow_mesh, output_path="output/cow-retexture.gif")

100%|██████████| 36/36 [00:31<00:00,  1.14it/s]


Result:

- Color 1: Blue
- Color 2: Red

![](output/cow-retexture.gif)

# 4. Camera Transformations (10 points)

In [2]:
import pytorch3d as p3d
import numpy as np
from starter.camera_transforms import render_textured_cow
from starter.utils import get_mesh_renderer
import matplotlib.pyplot as plt

R_relative = [
    [[1.0, 0.0, 0.0],
     [0.0, 1.0, 0.0],
     [0.0, 0.0, 1.0]],
    [[0.0, 1.0, 0.0],
     [-1.0, 0.0, 0.0],
     [0.0, 0.0, 1.0]],
    [[1.0, 0.0, 0.0],
     [0.0, 1.0, 0.0],
     [0.0, 0.0, 1.0]],
    [[1.0, 0.0, 0.0],
     [0.0, 1.0, 0.0],
     [0.0, 0.0, 1.0]],
    [[0.0, 0.0, 1.0],
     [0.0, 1.0, 0.0],
     [-1.0, 0.0, 0.0]],
]
T_relative = [
    [0.0, 0.0, 0.0],
    [0.0, 0.0, 0.0],
    [0.0, 0.0, 3.0],
    [0.5, -0.5, 0.0],
    [-3.0, 0.0, 3.0],
]

plt.ioff()
for i in range(len(R_relative)):
    image = render_textured_cow(cow_path="data/cow.obj", R_relative=R_relative[i], T_relative=T_relative[i])
    plt.imshow(image)
    plt.axis('off')
    plt.imsave(f"output/cow_view_{i}.png", image)
plt.ion()

R_relative and T_relative are another affine transformation pair that defines the camera's pose relative to the cow model.

The formula to compute the new camera extrinsics is:

R_relative and T_relative are another affine transformation pair that defines the camera's pose relative to the cow model.

The formula to compute the new camera extrinsics is:

$$
R_{\text{new}} = R_{\text{relative}}\,R_{\text{cow}} \\
T_{\text{new}} = R_{\text{relative}}\,T_{\text{cow}} + T_{\text{relative}}
$$

The points will be transformed from world coordinates to camera coordinates using the new extrinsics.

$$
P_{\text{camera}} = R_{\text{relative}} \cdot (R_{\text{cow}} \cdot P_{\text{world}} + T_{\text{cow}}) + T_{\text{relative}}
$$

Results:

![](output/cow_view_0.png) ![](output/cow_view_1.png) ![](output/cow_view_2.png) ![](output/cow_view_3.png) ![](output/cow_view_4.png) ![](output/cow_view_5.png)

# 5. Rendering Generic 3D Representations (45 Points)

In [16]:
from starter.utils import get_points_renderer
from tqdm import tqdm
import numpy as np
import imageio

def camera_point_360(point_cloud, img_size=256, num_frames=36, duration=1000, distance=3.0, elevation=15.0, output_path=None):
    points_renderer = get_points_renderer(
        image_size=img_size,
        radius=0.01,
    )

    duration //= num_frames  # duration per frame in milliseconds
    degrees = np.linspace(0, 360, num=num_frames).tolist()
    my_images = []

    for angle in tqdm(degrees):
        R, T = p3d.renderer.cameras.look_at_view_transform(distance, elevation, angle)
        cameras = p3d.renderer.FoVPerspectiveCameras(R=R, T=T, fov=60.0)
        rend = points_renderer(point_cloud, cameras=cameras)
        image = rend[0, ..., :3].numpy()  # (B, H, W, 4) -> (H, W, 3).
        image = np.flipud(image)
        my_images.append((image * 255).astype(np.uint8))

    if output_path is not None:
        imageio.mimsave(output_path, my_images, duration=duration, loop=0)
        return None
    else:
        return my_images

## 5.1 Rendering Point Clouds from RGB-D Images (10 points)

In [ ]:
import torch
import pytorch3d as p3d
from starter.render_generic import load_rgbd_data
from starter.utils import unproject_depth_image

data = load_rgbd_data()

points1, rgba1 = unproject_depth_image(
    torch.tensor(data['rgb1']),
    torch.tensor(data['mask1']),
    torch.tensor(data['depth1']),
    data['cameras1'],
)
points2, rgba2 = unproject_depth_image(
    torch.tensor(data['rgb2']),
    torch.tensor(data['mask2']),
    torch.tensor(data['depth2']),
    data['cameras2'],
)

point_cloud1 = p3d.structures.Pointclouds(
    points=[points1],
    features=[rgba1[:, :3]],
)
point_cloud2 = p3d.structures.Pointclouds(
    points=[points2],
    features=[rgba2[:, :3]],
)
point_cloud_unified = p3d.structures.Pointclouds(
    points=[torch.cat([points1, points2], dim=0)],
    features=[torch.cat([rgba1[:, :3], rgba2[:, :3]], dim=0)],
)

camera_point_360(point_cloud1, distance=6.0, elevation=0, output_path="output/point_cloud1_360.gif")
camera_point_360(point_cloud2, distance=6.0, elevation=0, output_path="output/point_cloud2_360.gif")
camera_point_360(point_cloud_unified, distance=6.0, elevation=0, output_path="output/point_cloud_unified_360.gif")

100%|██████████| 4/4 [02:10<00:00, 32.53s/it]


![](output/point_cloud1_360.gif) ![](output/point_cloud2_360.gif) ![](output/point_cloud_unified_360.gif)

## 5.2 Parametric Functions (10 + 5 points)

## 5.3 Implicit Surfaces (15 + 5 points)

# 6. Do Something Fun (10 points)

# (Extra Credit) 7. Sampling Points on Meshes (10 points)